Build knn-based cancer type classifier using gdc data.
Use the knn hyperparameters of the best classifier to draw the knn cell lines for PMDs.

**Dimensionality reduction:**<br>
- https://colah.github.io/posts/2014-10-Visualizing-MNIST/
- https://jlmelville.github.io/smallvis/mmds.html
- https://www.cs.toronto.edu/~hinton/csc2535/notes/lec11new.pdf

1. Input data
    - (we want dim-reduction methods that preserve distance proximity)
    - raw data lincs1000
    - pca
    - mds
    - sammon - emphesizes more the local rather the global structure
    - ae (up to 8 dims)
    - som (https://ieeexplore.ieee.org/document/5551813/) - preserves distance and proximity ()
2. Distance metric

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys

In [3]:
utils_path = os.path.abspath(os.path.join('..', 'utils_py'))
sys.path.append(utils_path)

In [4]:
from pilot1_imports import *
from utils import *
SEED = 0

In [5]:
DATAPATH = '/Users/apartin/work/jdacs/Benchmarks/Data/Pilot1'
DATASET = 'combined_rnaseq_data_lincs1000_combat'
# DATASET = 'combined_rnaseq_data_lincs1000_source_scale'
PDM_METADATA_FILENAME = 'combined_metadata_2018May.txt'

In [6]:
df_rna = load_combined_rnaseq(dataset=os.path.join(DATAPATH, DATASET), chunksize=2000, verbose=False)
print('\n{}'.format(df_rna.shape))

Loading dataframe by chunks...
/Users/apartin/work/jdacs/Benchmarks/Data/Pilot1/combined_rnaseq_data_lincs1000_combat: (15198, 943)

(15198, 943)


In [7]:
# Load metadata
meta = pd.read_csv(os.path.join(DATAPATH, PDM_METADATA_FILENAME), sep='\t')
meta = update_metadata_comb_may2018(meta)

In [8]:
# Extract dgc
meta = meta[meta['source']=='gdc'].reset_index(drop=True)
df_rna, meta = update_df_and_meta(df_rna, meta, on='Sample')
print(meta['source'].value_counts(dropna=False))
print(meta['ctype'].nunique())

gdc    11081
Name: source, dtype: int64
33


In [9]:
meta['ctype'].value_counts()[:5]

Breast Invasive Carcinoma               1222
Kidney Renal Clear Cell Carcinoma        608
Lung Adenocarcinoma                      594
Uterine Corpus Endometrial Carcinoma     587
Thyroid Carcinoma                        567
Name: ctype, dtype: int64

In [10]:
# Balance the dataset
df_rna, dropped_labels = balance_df(df=df_rna, y=meta['ctype'], min_label_size=50, seed=None)
df_rna, meta = update_df_and_meta(df_rna, meta, on='Sample')
print(df_rna.shape)

(1550, 943)


In [11]:
dropped_labels

{'Cholangiocarcinoma': 45,
 'Lymphoid Neoplasm Diffuse Large B-cell Lymphoma': 48}

In [12]:
meta['ctype'].value_counts()[:5]

Ovarian Serous Cystadenocarcinoma        50
Head and Neck Squamous Cell Carcinoma    50
Glioblastoma Multiforme                  50
Sarcoma                                  50
Kidney Renal Papillary Cell Carcinoma    50
Name: ctype, dtype: int64

### Utils

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import MDS
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix, f1_score

In [13]:
def plot_confusion_matrix(y_true, y_pred, labels=None, title=None, savefig=True, img_name='confusion'):
    """ Create a confusion matrix for a classification results.
    Args:
        labels : list of label names
    """
    np_conf = confusion_matrix(y_true, y_pred)
    df_conf = pd.DataFrame(np_conf, index=labels, columns=labels)

    m = df_conf.shape[0]

    fontsize=25  # font size of labels (not in table numbers)
    plt.figure(figsize=(m, m))
    sns.set(font_scale=2.0)
    sns.heatmap(df_conf, annot=True, fmt='d', linewidths=0.9, cmap='Greens', linecolor='white')
    plt.ylabel('True label', fontsize=fontsize)
    plt.xlabel('Predicted label', fontsize=fontsize)
    if title:
        plt.title(title, fontsize=fontsize)

#     if savefig:
#         plt.savefig(img_name, bbox_inches='tight')

    return df_conf

In [14]:
sample_vec = df_rna['Sample']
rna = df_rna.iloc[:, 1:].copy()

In [15]:
xtr, xte, ytr, yte = train_test_split(rna, meta['ctype'], test_size=0.2,
                                      stratify=meta['ctype'], random_state=SEED, shuffle=True)
xtr.reset_index(drop=True, inplace=True)
xte.reset_index(drop=True, inplace=True)
ytr.reset_index(drop=True, inplace=True)
yte.reset_index(drop=True, inplace=True)
y_true_classes = yte.values

In [16]:
# yte.value_counts()

In [17]:
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='brute', 
                                 metric='minkowski', p=2, metric_params=None, n_jobs=-1);
knn_model.fit(xtr, ytr)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [18]:
y_pred_classes = knn_model.predict(xte)

In [19]:
precision_recall_fscore_support(yte, y_pred_classes, average='micro')

(0.7811447811447811, 0.7811447811447811, 0.7811447811447811, None)

In [20]:
labels = (np.unique(np.append(arr=y_pred_classes, values=y_true_classes))).tolist()
print(len(labels))

33


In [21]:
confusion_matrix(y_true_classes, y_pred_classes, labels=labels)

array([[9, 0, 0, ..., 0, 0, 0],
       [0, 8, 0, ..., 1, 0, 0],
       [0, 0, 5, ..., 2, 0, 0],
       ...,
       [0, 0, 0, ..., 8, 1, 0],
       [0, 0, 0, ..., 2, 5, 0],
       [0, 0, 0, ..., 0, 0, 9]])

In [22]:
# plot_confusion_matrix(y_true=y_true_classes, y_pred=y_pred_classes, labels=labels);

### Dim-reduction

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import MDS
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix, f1_score

In [24]:
# from minisom import MiniSom

In [25]:
sample_vec = df_rna['Sample']
rna = df_rna.iloc[:, 1:].copy()

#### Create different classifiers

In [26]:
# Create different classifier
classifiers = [('minkowski (p=2)', KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='brute', 
                                                        metric='minkowski', p=2, metric_params=None, n_jobs=-1)),
               ('minkowski (p=1)', KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='brute', 
                                                        metric='minkowski', p=1, metric_params=None, n_jobs=-1)),
               ('chebyshev', KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='brute', 
                                                  metric='chebyshev', metric_params=None, n_jobs=-1)),
               ('cosine', KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='brute', 
                                               metric='cosine', metric_params=None, n_jobs=-1)),
              ]

#### Create different datasets

In [27]:
n_components=8

In [28]:
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
pca_obj = PCA(n_components=n_components, copy=True, whiten=False, svd_solver='auto', tol=0.0,
              iterated_power='auto', random_state=SEED)
rna_pca = pca_obj.fit_transform(rna.copy())
print(rna_pca.shape)

(1485, 8)


In [29]:
pca_obj.explained_variance_ratio_.sum()

0.5919773477096065

In [30]:
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html
ica_obj = FastICA(n_components=n_components, algorithm='parallel', whiten=True, fun='logcosh',
                  fun_args=None, max_iter=800, tol=0.001, w_init=None, random_state=SEED)
rna_ica = ica_obj.fit_transform(rna.copy())
print(rna_ica.shape)

(1485, 8)


In [31]:
# # http://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html
# mds_obj = MDS(n_components=2, metric=True, n_init=1, max_iter=100,
#               verbose=1, eps=0.001, n_jobs=-1, random_state=SEED, dissimilarity='euclidean')
# rna_mds = mds_obj.fit_transform(df_rna.copy())
# print(rna_mds.shape)

In [32]:
# Create different classifier
datasets = [('original', rna.values), ('pca', rna_pca), ('ica', rna_ica)]

#### Iterate

In [33]:
for ci, c in enumerate(classifiers):
    # Iter over classifiers
    cls_name = classifiers[ci][0]
    cls_obj = classifiers[ci][1]
    print('\n', cls_name)
    print('----------------')
    
    for di, d in enumerate(datasets):
        # Iter over datasets
        data_name = datasets[di][0]
        data = datasets[di][1].copy()
        print('  ', data_name)
        
        # Split dataset
        xtr, xte, ytr, yte = train_test_split(data, meta['ctype'], test_size=0.2,
                                              stratify=meta['ctype'], random_state=SEED, shuffle=True)
        y_true_classes = yte.values
        
        # Train kNN
        cls_obj.fit(xtr, ytr)
        
        # Compute class predictions
        y_pred_classes = cls_obj.predict(xte)
        
        # Compute scores
        scores = precision_recall_fscore_support(yte, y_pred_classes, average='micro')
        print('     precision={:.2f}, recall={:.2f}, f_beta={:.2f}'.format(scores[0], scores[1], scores[2]))
        


 minkowski (p=2)
----------------
   original
     precision=0.78, recall=0.78, f_beta=0.78
   pca
     precision=0.62, recall=0.62, f_beta=0.62
   ica
     precision=0.69, recall=0.69, f_beta=0.69

 minkowski (p=1)
----------------
   original
     precision=0.77, recall=0.77, f_beta=0.77
   pca
     precision=0.63, recall=0.63, f_beta=0.63
   ica
     precision=0.69, recall=0.69, f_beta=0.69

 chebyshev
----------------
   original
     precision=0.74, recall=0.74, f_beta=0.74
   pca
     precision=0.58, recall=0.58, f_beta=0.58
   ica
     precision=0.66, recall=0.66, f_beta=0.66

 cosine
----------------
   original
     precision=0.82, recall=0.82, f_beta=0.82
   pca
     precision=0.64, recall=0.64, f_beta=0.64
   ica
     precision=0.67, recall=0.67, f_beta=0.67
